# --> Importations

In [106]:
import tensorflow as tf
import numpy as np

# --> Creation d'un model simple

In [107]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

# --> Setup optimizer, loss, metrics

In [108]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() #Loss function
optimizer = tf.keras.optimizers.Adam()                        #Optimizer
train_loss = tf.keras.metrics.Mean(name='train_loss')         #Track l'evolution du model avec l'accumulateur

# --> Definir l'operation d'entrainement

In [109]:
@tf.function
def train_step(image, targets):
    #with tf.GradientTape() permet de calculer toutes les operations qui ont ete faite
    #Ce qu'il y a a l'interieur de l'indexation est capture pour ensuite calculer
    #le gradient.
    with tf.GradientTape() as tape:
        predictions = model(image)               #"predictions" est un tensor
        loss = loss_object(targets, predictions) #pour pouvoir utiliser loss_object().
    #On calcule le gradient avec les variables accessible par model.trainable_variables.
    #On recupere aussi le loss pour le calculer.
    gradients = tape.gradient(loss, model.trainable_variables) 
    for gradient in gradients:
        print("Gradients shape : ", gradient.shape) #(30, 256) Il y a 256 neurones sur la couche intermediaire et 30 entrees.
                                                    #(256,) Valeurs des biais de l'hidden layer.
                                                    #(256, 128) Il y a 256 neurones sur la couche intermediaire et 128 .
                                                    #sur la couche intermediaire 2.
                                                    #(128,) Valeurs des biais de l'hidden layer 2.
                                                    #(128, 2) Il y a 128 neurones sur la couche intermediaire 2 et 2 sorties.
                                                    #(2,) Valeurs des biais de la layer de sortie .
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) #On applique le gradient a l'optimizer Adam.
                                                                         #zip() permet de faire l'association entre les
                                                                         #gradients et les variables.
    train_loss(loss) #On continue de tracker l'evolution du loss.

# --> Generation de fausse donnees

In [116]:
for epoch in range(0, 10):
    for _ in range(0, 10):
        #On cree des faux inputs
        inputs = np.zeros((2, 30))
        inputs[0] -= 1
        inputs[1] = 1
        #On cree les faux targets
        targets = np.zeros((2, 1))
        targets[0] = 0
        targets[1] = 1
        #On train le model avec ces fausses donnees
        train_step(inputs, targets)
    #train_loss(0.0001) #On peut ajouter des valeurs pour verifier que le result() fonctionne bien
    print("Loss : %s" % train_loss.result()) #Affiche la moyenne de toutes les erreurs stockees a l'interieur de train_loss
    train_loss.reset_states() #Vide le buffer

Loss : tf.Tensor(2.5963353e-05, shape=(), dtype=float32)
Loss : tf.Tensor(2.433619e-05, shape=(), dtype=float32)
Loss : tf.Tensor(2.2858034e-05, shape=(), dtype=float32)
Loss : tf.Tensor(2.1522927e-05, shape=(), dtype=float32)
Loss : tf.Tensor(2.0330863e-05, shape=(), dtype=float32)
Loss : tf.Tensor(1.9234163e-05, shape=(), dtype=float32)
Loss : tf.Tensor(1.822091e-05, shape=(), dtype=float32)
Loss : tf.Tensor(1.7308976e-05, shape=(), dtype=float32)
Loss : tf.Tensor(1.647453e-05, shape=(), dtype=float32)
Loss : tf.Tensor(1.5657963e-05, shape=(), dtype=float32)


# --> Important

In [117]:
try:
    input_ = np.zeros((1, 30)) + 1
    print(model.predict(input_))
except Exception as e:
    print("Error : ", e)

[[5.0665835e-06 9.9999487e-01]]


In [118]:
input_ = np.zeros((1, 30)) + 1
print(model(input_))

tf.Tensor([[5.0665835e-06 9.9999487e-01]], shape=(1, 2), dtype=float32)
